In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt
import ta
import yfinance as yf
import seaborn as sns
import numpy as np
from queue import Queue
import holidays
import scipy as sp
from itertools import combinations
from pymongo import MongoClient
import swifter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

C:\Users\thede\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from helper.client import *
from helper.machinelearner import *
from helper.dbmgr import *
from helper.dataprocessor import *

# Create an instance of CoinbaseClient
client = CoinbaseClient(api_key, api_secret)

In [3]:
db_uri = 'mongodb://localhost:27017/'  
db_name = 'crypto_data'  

# Get and format data from CB

In [4]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

print(x.inserted_id)

65e33f009cf514fa37126215


In [5]:
unique_pids = client.get_unique_product_ids()
unique_pids = [f"{i}-{'USDC'}" for i in unique_pids]

# start_time = datetime(2022, 10, 1)
start_time = datetime(2024, 1, 1)
end_time = datetime(2024, 2, 29)

db_manager = DatabaseManager(start_time=start_time, end_time=end_time)
print(db_manager.check_row_counts())

db_manager.main(unique_pids)

Checking the number of documents in each collection...
Collection 'crypto_candles_fifteen_minute' has 0 documents.
Collection 'crypto_candles_thirty_minute' has 0 documents.
Collection 'crypto_candles_one_hour' has 0 documents.
Collection 'crypto_candles_two_hour' has 0 documents.
Collection 'crypto_candles_six_hour' has 0 documents.
Collection 'crypto_candles_one_day' has 0 documents.
None
Preparing to fetch and store data...


# Technical Indicator creation

In [ ]:
fe = FeatureEngineer(db_name)
processed_data = fe.run()

Processing Collections: 100%|██████████| 6/6 [01:33<00:00, 15.61s/it]


# Preprocessing creation
- Labels
- Scaling
- Normalisation
- Stationary features

In [ ]:
data_preparator = DataPreparator(db_name)

# Prepare the data
final_data, non_stationary_columns = data_preparator.prepare_data()

In [ ]:
### label generation

label_generator = LabelGenerator(mongo_uri='mongodb://localhost:27017/',
                                  db_name='crypto_data', account_value=5000,
                                    tx_fee=0.006, profit_level=0.003)

label_generator = LabelGenerator(mongo_uri='mongodb://localhost:27017/', db_name='crypto_data', account_value=5000, tx_fee=0.006, profit_level=0.003)

# Specify the collection name where you want to save the labeled data
collection_name = 'labeled_crypto_data'

# Get the list of instruments from the database
instruments = label_generator.get_instruments()

# Iterate through each instrument and generate labels
for instrument in instruments:
    # Fetch data for the instrument
    df = label_generator.fetch_data(instrument)

    # Check if there's enough data to process
    if not df.empty:
        # Process the data to generate labels for this specific instrument
        for granularity in label_generator.granularities:
            df_granularity = df[df['granularity'] == granularity]

            for window_size in [15, 30, 45, 60]:
                if not df_granularity.empty:
                    result_df = label_generator.calculate_targets(df_granularity.copy(), window_size)
                    result_df['instrument'] = instrument
                    result_df['granularity'] = granularity
                    result_df['window_size'] = window_size

                    # Save the labeled data for this instrument to the MongoDB collection
                    label_generator.save_to_database(result_df, collection_name)

                    # Optional: Print a message to confirm data has been saved
                    print(f"Saved labeled data for {instrument} with granularity {granularity} and window size {window_size} to {collection_name}")


# Machine Learning on one set of parameters
- Grid Search for best parameters
- view feature importance on the best set and trim

In [ ]:
mongo_uri = 'mongodb://localhost:27017/'  # Adjust as necessary
db_name = 'crypto_data'  # Your database name
collection_name = 'labeled_crypto_data'  # Your collection name
target_column = 'target15'  # Your target column name

trainer = MongoDBMLModelTrainer(mongo_uri, db_name, collection_name, target_column)
trainer.train_and_evaluate()

# Save best model for backtesting

# Backtester